In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/hackaton_students_train.csv')

In [3]:
data['y'] = (
    data['graphic_design'].astype(str) +
    data['top_1_section'].astype(str) +
    data['top_2_section'].astype(str) +
    data['top_3_section'].astype(str)   
)
data

,id,booking_adult_count,booking_child_count,booking_destination_airport_code,booking_id,booking_infant_count,booking_leg_count,booking_market,booking_origin_airport_code,booking_original_currency,...,pnr,preffered_language,request_dttm,request_id,clicked,graphic_design,top_1_section,top_2_section,top_3_section,y
0,105529,1,0,TBS,6386840,0,2,GE,WAW,GEL,...,40faefe514be47b38cea3be065583d7f,en_gb,2014-03-02T21:13:05.030Z,ee37548ce2a74b99b41d67f4d2a138fd,0,C,6,3,0,C630
1,28386,1,0,WAW,6842736,0,2,US,EWR,USD,...,7fafefddde544aba95c69552cfbadd51,en_gb,2013-10-09T11:08:14.112Z,f083457969dd46d2ada761198096ecc4,1,B,4,3,5,B435
2,107663,1,0,BEG,3164316,0,2,PL,SZZ,NaN,...,6240c081718e4439a3174ac38a2c9c45,en_gb,2014-03-08T08:15:38.344Z,9df9a894f4374dfba0d8811dd99c1b1e,0,C,3,6,0,C360
3,24881,1,0,BOM,3835074,0,1,SE,WAW,NaN,...,231c99e3d0644366827e91276296e3cc,en_gb,2013-10-06T09:09:48.597Z,89c95fa040db42d4a1982c7901128585,1,A,3,1,2,A312
4,90383,1,0,OTP,5962948,0,2,PL,OSL,PLN,...,4854301527cf48f7a32c9625a154874b,en_gb,2014-01-07T14:14:06.568Z,d44b65a240374e61867039b37c98f9a0,1,B,3,4,5,B345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84944,7634,1,0,WAW,5494177,0,2,US,EWR,USD,...,d86c510a2b114288bc6ca23cbdb20731,pl_pl,2013-09-21T08:09:47.626Z,c96bd1b541614661a3ec092f4594e5c1,1,B,4,3,5,B435
84945,67152,1,0,ICN,4024574,0,2,PL,WRO,PLN,...,882f81dc62264d18be14b0c346249888,en_gb,2013-11-16T11:11:36.066Z,22e4542ce44049da8a6f20ed6238faf0,1,B,3,4,5,B345
84946,94614,2,0,DEL,9014936,0,2,PL,WAW,PLN,...,828a20846c724728ac8a0760efdf6df6,pl_pl,2014-01-27T22:12:08.965Z,2532b32403f2417fb5717281ddadb9df,1,C,3,6,0,C360
84947,1031,2,0,DBV,5195800,0,2,PL,WAW,PLN,...,0e0f7bb498134733a28aae066201993d,pl_pl,2013-09-15T12:34:09.192Z,e5d4b884d3f24c54a47162af74a1ad09,0,A,3,2,1,A321


In [4]:
data = data.drop(columns=['graphic_design', 'top_1_section', 'top_2_section', 'top_3_section'])



In [5]:
from L_score import L_score
import numpy as np
y_true = data[['clicked', 'y']]
y_true

y_pred = data['y'].copy().to_numpy()
# shuffle
y_pred[1:10000] = 'C340'

l = L_score(y_true, y_pred)
l

0.8809763101220388